#### Lets dismantle the tools

Tools are interfaces that an agent can use to interact with the world.

> The name of the tool

> A description of what the tool is

> JSON schema of what the inputs to the tool are

> The function to call

> Whether the result of a tool should be returned directly to the user

To dismantle and understand the tools, we need to learn how agents are integrating with Agents. Lets dive into Tools First

pip install langchain langchain-openai beautifulsoup4 wikipedia langchainhub langserve[all] langchain-community python-dotenv tavily-python


In [1]:
from langchain_community import (
    tools,
    agent_toolkits,
)

In [10]:
tools.__all__

['AINAppOps',
 'AINOwnerOps',
 'AINRuleOps',
 'AINTransfer',
 'AINValueOps',
 'AIPluginTool',
 'APIOperation',
 'ArxivQueryRun',
 'AzureCogsFormRecognizerTool',
 'AzureCogsImageAnalysisTool',
 'AzureCogsSpeech2TextTool',
 'AzureCogsText2SpeechTool',
 'AzureCogsTextAnalyticsHealthTool',
 'BaseGraphQLTool',
 'BaseRequestsTool',
 'BaseSQLDatabaseTool',
 'BaseSparkSQLTool',
 'BaseTool',
 'BearlyInterpreterTool',
 'BingSearchResults',
 'BingSearchRun',
 'BraveSearch',
 'ClickTool',
 'CogniswitchKnowledgeRequest',
 'CogniswitchKnowledgeSourceFile',
 'CogniswitchKnowledgeSourceURL',
 'CogniswitchKnowledgeStatus',
 'ConneryAction',
 'CopyFileTool',
 'CurrentWebPageTool',
 'DeleteFileTool',
 'DuckDuckGoSearchResults',
 'DuckDuckGoSearchRun',
 'E2BDataAnalysisTool',
 'EdenAiExplicitImageTool',
 'EdenAiObjectDetectionTool',
 'EdenAiParsingIDTool',
 'EdenAiParsingInvoiceTool',
 'EdenAiSpeechToTextTool',
 'EdenAiTextModerationTool',
 'EdenAiTextToSpeechTool',
 'EdenaiTool',
 'ElevenLabsText2SpeechT

In [3]:
len(tools.__all__)

131

In [3]:
agent_toolkits.__all__

['AINetworkToolkit',
 'AmadeusToolkit',
 'AzureCognitiveServicesToolkit',
 'CogniswitchToolkit',
 'ConneryToolkit',
 'FileManagementToolkit',
 'GmailToolkit',
 'JiraToolkit',
 'JsonToolkit',
 'MultionToolkit',
 'NLAToolkit',
 'NasaToolkit',
 'O365Toolkit',
 'OpenAPIToolkit',
 'PlayWrightBrowserToolkit',
 'PolygonToolkit',
 'PowerBIToolkit',
 'SQLDatabaseToolkit',
 'SlackToolkit',
 'SparkSQLToolkit',
 'SteamToolkit',
 'ZapierToolkit',
 'create_json_agent',
 'create_openapi_agent',
 'create_pbi_agent',
 'create_pbi_chat_agent',
 'create_spark_sql_agent',
 'create_sql_agent']

In [6]:
len(agent_toolkits.__all__)

28

In [32]:
from langchain_community.tools import (
    WikipediaQueryRun,
)
from langchain_community.utilities import (
    WikipediaAPIWrapper
)

In [19]:
from inspect import getsource
print(getsource(WikipediaQueryRun))

class WikipediaQueryRun(BaseTool):
    """Tool that searches the Wikipedia API."""

    name: str = "wikipedia"
    description: str = (
        "A wrapper around Wikipedia. "
        "Useful for when you need to answer general questions about "
        "people, places, companies, facts, historical events, or other subjects. "
        "Input should be a search query."
    )
    api_wrapper: WikipediaAPIWrapper

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Wikipedia tool."""
        return self.api_wrapper.run(query)



In [7]:
print(getsource(WikipediaAPIWrapper))

class WikipediaAPIWrapper(BaseModel):
    """Wrapper around WikipediaAPI.

    To use, you should have the ``wikipedia`` python package installed.
    This wrapper will use the Wikipedia API to conduct searches and
    fetch page summaries. By default, it will return the page summaries
    of the top-k results.
    It limits the Document content by doc_content_chars_max.
    """

    wiki_client: Any  #: :meta private:
    top_k_results: int = 3
    lang: str = "en"
    load_all_available_meta: bool = False
    doc_content_chars_max: int = 4000

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the python package exists in environment."""
        try:
            import wikipedia

            wikipedia.set_lang(values["lang"])
            values["wiki_client"] = wikipedia
        except ImportError:
            raise ImportError(
                "Could not import wikipedia python package. "
                "Please install it with `p

In [20]:
api_client = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=150)
wiki_tool = WikipediaQueryRun(api_wrapper=api_client)

In [29]:
print(wiki_tool.name)
print(wiki_tool.description)
print(wiki_tool.return_direct)
print(wiki_tool.args)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
False
{'query': {'title': 'Query', 'type': 'string'}}


In [30]:
wiki_tool.run("Interstellar")

d:\tempVids\onlylc\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file d:\tempVids\onlylc\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


'No good Wikipedia Search Result was found'

In [2]:
# Agents can wield tools, and these can be custom built also

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [3]:
@tool
def search(query: str) -> str:
    """lookup results online"""
    return 'This is search return'

In [4]:
@tool
def multiply(a: int, b: int) -> str:
    """Multiply two numbers"""
    return a * b 

In [5]:
print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - lookup results online
{'query': {'title': 'Query', 'type': 'string'}}


In [15]:
search('Hello')

'This is search return'

In [6]:
# sub-classing BaseTool
class CalculatorInput(BaseModel):
    a: int = Field(description='A number to get product')
    b: int = Field(description='Another number to get product')

In [7]:
class CustomCalculatorTool(BaseTool):
    name = "calculator"
    description = "useful class that helps in math calculations"
    args_schema = CalculatorInput
    return_direct = True

    def _run(self, a, b, run_manager):
        """Use the tool"""
        return a * b
    
    async def _arun(self, a, b, run_manager):
        """Use the tool only Asynchronously"""
        raise NotImplementedError("Calc doesnt support Async")

In [8]:
cust_calc = CustomCalculatorTool()

In [9]:
print(cust_calc.name)
print(cust_calc.description)
print(cust_calc.args)
print(cust_calc.return_direct)

calculator
useful class that helps in math calculations
{'a': {'title': 'A'}, 'b': {'title': 'B'}}
True


In [10]:
# Take a function and make it to tool using structured tool

def make_name(fname):
    return f"my_file_{fname}.json"


name_tool = StructuredTool.from_function(
    func=make_name,
    name="Make filename",
    description="Tool that makes a file name",
    handle_tool_error=True
)

In [11]:
name_tool.run(tool_input='SuperNice')

'my_file_SuperNice.json'

In [12]:
from dotenv import load_dotenv
load_dotenv("D:\\gitFolders\\python_de_learners_data\\.env")

True

In [13]:
from langchain_openai import ChatOpenAI, OpenAI
import os
llm = ChatOpenAI(model='gpt-3.5-turbo',
                 api_key=os.environ['OPENAI_API_KEY'])

In [7]:
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x000001E764A1B290> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001E7649EE050> openai_api_key=SecretStr('**********') openai_proxy=''


In [14]:
from langchain_core.utils.function_calling import convert_to_openai_function
openai_nametool = convert_to_openai_function(make_name)
openai_nametool

{'name': 'make_name',
 'description': '',
 'parameters': {'type': 'object', 'properties': {}, 'required': ['fname']}}

In [21]:
wiki_openai = convert_to_openai_function(wiki_tool)
wiki_openai

{'name': 'wikipedia',
 'description': 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.',
 'parameters': {'properties': {'__arg1': {'title': '__arg1',
    'type': 'string'}},
  'required': ['__arg1'],
  'type': 'object'}}

In [22]:
from langchain_core.messages import HumanMessage
message = llm.invoke([HumanMessage("Get wiki results")],
                     functions=[wiki_openai])

In [24]:
text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text,id='a')]

In [ ]:
messages[0].dict()

In [25]:
llm.invoke(messages)

AIMessage(content='Rainbow Toes Co.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 22, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

#### Diving into Agents

AgentAction: Represents Action to be taken, with a tool and tool_input property

AgentFinish: Represents final results from an agent, contain return_values (dict) which contains 'output' key

Intermediate Steps: Represents 'Prev actions' & corresponding outputs from this Current Agent Run.

Agent : Is a Chain 

LLM --> Prompt --> Output Parser

AgentInput: KV mapping which requires 'Intermediate_steps' key

AgentOutput: Next Action to take or Final Outputs to send to User (AgentAction, 
AgentFinish)

Main thing affects Agent is the prompting strategy used

The goal of the OpenAI tools APIs is to more reliably return valid and useful function calls than what can be done using a generic text completion or chat API.

In [36]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.tools.tavily_search import TavilySearchResults

In [37]:
tools = [TavilySearchResults(max_results=1)]

In [28]:
tools[0].name

'tavily_search_results_json'

In [35]:
tools[0].run(tool_input="Where is moon",
             color='yellow',
             run_name='tester')

[{'url': 'https://science.nasa.gov/moon/facts/',
  'content': "Water on the Moon\nDuring the initial exploration of the Moon, and the analysis of all the returned samples from the Apollo and the Luna missions, we thought that the surface of the Moon was dry.\n Discover More Topics From NASA\nSun\nPlanets\nAsteroids, Comets & Meteors\nKuiper Belt\nThe National Aeronautics and Space Administration\nNASA explores the unknown in air and space, innovates for the benefit of humanity, and inspires the world through discovery.\n While you were there, you'd notice that the gravity on the surface of the Moon is one-sixth of Earth's, which is why in footage of moonwalks, astronauts appear to almost bounce across the surface.\n Missions such as Lunar Prospector, LCROSS, and Lunar Reconnaissance Orbiter, have not only shown that the surface of the Moon has global hydration but there are actually high concentrations of ice water in the permanently shadowed regions of the lunar poles.\n The discovery

In [39]:
myagent = create_openai_tools_agent(llm, tools, prompt)

In [13]:
myagent.get_prompts()

[ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlacehol

In [16]:
myagent.get_name()

'RunnableSequence'

In [17]:
myagent.get_graph()

Graph(nodes={'7ca6493e1020495b9d23ab454f3ef663': Node(id='7ca6493e1020495b9d23ab454f3ef663', data=<class 'pydantic.v1.main.RunnableParallel<agent_scratchpad>Input'>), '8c27ce82eb1241eda4913011b29bebb9': Node(id='8c27ce82eb1241eda4913011b29bebb9', data=<class 'pydantic.v1.main.RunnableParallel<agent_scratchpad>Output'>), 'e316ff9aeb5c4ce382b128e32ce46371': Node(id='e316ff9aeb5c4ce382b128e32ce46371', data=RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))), '5429921429aa47f28434cf5c839f30b9': Node(id='5429921429aa47f28434cf5c839f30b9', data=RunnablePassthrough()), '913c40c7eaee428f984db3bb358bda2f': Node(id='913c40c7eaee428f984db3bb358bda2f', data=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.f

In [35]:
myagent.get_prompts()[0].metadata

{'lc_hub_owner': 'hwchase17',
 'lc_hub_repo': 'openai-tools-agent',
 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}

In [37]:
myagent.get_prompts()[0].messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [19]:
myagent.lc_id()

['langchain', 'schema', 'runnable', 'RunnableSequence']

In [40]:
agent_execution_env = AgentExecutor(agent=myagent,
                                    tools=tools, verbose=True)


In [33]:
agent_execution_env.get_prompts()

[]

In [48]:
agent_execution_env.lookup_tool('tavily_search_results_json')

TavilySearchResults(max_results=1)

In [30]:
# agent_execution_env.agent.save('newagent.agent')
# agent_execution_env.save_agent('newagent.agent')
# agent_execution_env.update_forward_refs()

In [49]:
type(agent_execution_env)

langchain.agents.agent.AgentExecutor

In [41]:
# The AgentExecutor class doesn't have invoke method, so where is the invoke coming into picture
# if you follow with invoke method below, it will go to lc/chains/base.py /Chain class
# The above Chain class is ABC, so need to find which kind of agent_chain is working when calling AgentExecutor
# The AgentExecutor itself is completely abstracted out, and trying to understand where it is origin was a challenge
# Decided to use the VsCode search option
# Then located the below "Invoking: " inside the openai_tools.py and openai_multi_agent/base.py
# format_scratchpad/log_to_messages.py contains the implementation of agent_scratchpad

agent_execution_env.invoke({"input": "When will the next near earth comet will arrive"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'next near earth comet arrival date'}`


[{'url': 'https://www.space.com/comet-coming-2024-could-be-bright', 'content': 'Such ices vaporize far from the sun, giving a distant comet a short-lived surge in brightness that in turn, raises unrealistic expectations.\nAlready been "around the block"\nIf, on the other hand, the latest calculations showed that Tsuchinshan–ATLAS was traveling in an elliptical orbit and returning to the sun from the distant past, its coating of highly volatile materials would have already been shed, and what we are now seeing is the true underlying level of activity. More likely a dud\nUnfortunately, it would seem more likely that Tsuchinshan–ATLAS, like Kohoutek and other comets with a similar lineage (Cunningham in 1940-41, Austin in 1990 and ISON in 2013), could ultimately fizzle, because it is a "new" comet coming out of the Oort cloud, the spherical shell surroun

{'input': 'When will the next near earth comet will arrive',
 'output': 'The next near-Earth comet, Comet Tsuchinshan–ATLAS, is expected to pass closest to Earth at a distance of 44 million miles (71 million km) between October 12 and October 19, 2024. If it lives up to its most optimistic expectations, it should put on its best show during this time. You can read more about it [here](https://www.space.com/comet-coming-2024-could-be-bright).'}

In [41]:
print(agent_execution_env.tools)

[TavilySearchResults(max_results=1)]


In [42]:
xml_prompt = hub.pull("hwchase17/xml-agent-convo")
print(xml_prompt.input_variables)
print(xml_prompt.metadata)
print(xml_prompt.messages[0].prompt.template)

['agent_scratchpad', 'input', 'tools']
{'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'xml-agent-convo', 'lc_hub_commit_hash': '00f6b7470fa25a24eef6e4e3c1e44ba07189f3e91c4d987223ad232490673be8'}
You are a helpful assistant. Help the user answer any questions.

You have access to the following tools:

{tools}

In order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>
For example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:

<tool>search</tool><tool_input>weather in SF</tool_input>
<observation>64 degrees</observation>

When you are done, respond with a final answer between <final_answer></final_answer>. For example:

<final_answer>The weather in SF is 64 degrees</final_answer>

Begin!

Previous Conversation:
{chat_history}

Question: {input}
{agent_scratchpad}


In [42]:
# Get the prompt to use - you can modify this!
jsonprompt = hub.pull("hwchase17/react-chat-json")
print(jsonprompt.input_variables)
print(jsonprompt.metadata)
print(jsonprompt.messages[0].prompt.template)

['agent_scratchpad', 'input', 'tool_names', 'tools']
{'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react-chat-json', 'lc_hub_commit_hash': '9c1258e8aa8ce33bebbd62e077c143d0b06c81f3c7de732187ee61c70c1254c7'}
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on th

In [43]:
from langchain.agents import create_json_chat_agent

json_agent = create_json_chat_agent(llm, tools, jsonprompt)

input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} partial_variables={'tools': 'tavily_search_results_json: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'tool_names': 'tavily_search_results_json'} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_re

In [45]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=json_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [46]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
```json
{
    "action": "tavily_search_results_json",
    "action_input": "LangChain"
}
```[{'url': 'https://en.wikipedia.org/wiki/LangChain', 'content': 'In October 2023 LangChain introduced LangServe, a deployment tool designed to facilitate the transition from LCEL (LangChain Expression Language) prototypes to production-ready applications.[5]\nIntegrations[edit]\nAs of March 2023, LangChain included integrations with systems including Amazon, Google, and Microsoft Azure cloud storage; API wrappers for news, movie information, and weather; Bash for summarization, syntax and semantics checking, and execution of shell scripts; multiple web scraping subsystems and templates; few-shot learning prompt generation support; finding and summarizing "todo" tasks in code; Google Drive documents, spreadsheets, and presentations summarization, extraction, and creation; Google Search and Microsoft Bing web search; OpenAI, Anthropic, and Hugging Face languag

{'input': 'what is LangChain?',
 'output': 'LangChain is a deployment tool introduced in October 2023 that facilitates the transition from LCEL prototypes to production-ready applications. It includes integrations with various systems such as cloud storage services, API wrappers for news, movie information, and weather, Bash for summarization and execution of shell scripts, web scraping subsystems, few-shot learning prompt generation support, and more. LangChain was launched in October 2022 as an open source project and quickly gained popularity, with significant contributions and funding. It is used for document analysis and summarization, chatbots, and code analysis.'}

In [72]:
# Get the prompt to use - you can modify this!
structured_prompt = hub.pull("hwchase17/structured-chat-agent")

In [79]:
print(structured_prompt.messages[0].prompt.template)

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation


In [80]:
# Get the prompt to use - you can modify this!
react_prompt = hub.pull("hwchase17/react")

In [83]:
print(react_prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [87]:
from langchain.agents import AgentExecutor, create_react_agent
tools = [TavilySearchResults(max_results=1)]
agent = create_react_agent(llm, tools, react_prompt)

In [88]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': 'tavily_search_results_json: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'tool_names': 'tavily_search_results_json'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this 

In [90]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

tools = [get_word_length]

In [91]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [92]:
llm_with_tools = llm.bind_tools(tools)

In [93]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [94]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent.save("testing_save.agent")

#### Understanding LCEL

In [95]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

"Why don't ice creams ever get stressed?\n\nBecause they always know how to chill out!"

In [98]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [101]:
output_parser.input_schema.schema()

{'title': 'StrOutputParserInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'name': {'title': 'Name', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'example': {'title': 'Example', 'default':

In [99]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'strin

In [100]:
chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}

In [102]:
model.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'name': {'title': 'Name', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}